In [23]:
from __future__ import print_function
from data import ImageDetectionsField, TextField, RawField
from data import COCO, DataLoader

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
import numpy as np
from time import time
import os.path


class objectview(object):
    def __init__(self, d):
        self.__dict__ = d
        
cfg = {'exp_name': 'transformer', 
       'batch_size' : 200 , 
       'workers' : 2, 
       'head' : 8, 
       'warmup' : 10000, 
       'resume_last' : False, 'resume_best' : False,
       'features_path' : './data/coco_detections.hdf5',
       'annotation_folder' : './data/annotations/',
       'logs_folder' : './tensorboard_logs',
       'N_enc' : 1,
       'N_dec' : 1,
       'use_amp' : True,
       'cuda' : True
      }

args = objectview(cfg)

# test_data_root = os.environ['DALI_EXTRA_PATH']
# file_root = os.path.join(test_data_root, 'db', 'coco', 'images')

file_root = args.features_path

# annotations_file = os.path.join(test_data_root, 'db', 'coco', 'instances.json')

annotations_file = os.path.join(args.annotation_folder, 'captions_train2014.json')

num_gpus = 1
batch_size = 16

In [24]:
class COCOPipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id):
        super(COCOPipeline, self).__init__(
            batch_size, num_threads, device_id, seed=15)
        self.input = ops.COCOReader(
            file_root=file_root,
            annotations_file=annotations_file,
            shard_id=device_id,
            num_shards=num_gpus,
            ratio=True,
            ltrb=True)
        self.decode = ops.ImageDecoder(device="mixed", output_type=types.RGB)
        self.flip = ops.Flip(device="gpu")
        self.bbflip = ops.BbFlip(device="cpu", ltrb=True)
        self.paste_pos = ops.Uniform(range=(0, 1))
        self.paste_ratio = ops.Uniform(range=(1, 2))
        self.coin = ops.CoinFlip(probability=0.5)
        self.coin2 = ops.CoinFlip(probability=0.5)
        self.paste = ops.Paste(device="gpu", fill_value=(32, 64, 128))
        self.bbpaste = ops.BBoxPaste(device="cpu", ltrb=True)
        self.prospective_crop = ops.RandomBBoxCrop(
            device="cpu",
            aspect_ratio=[0.5, 2.0],
            thresholds=[0.1, 0.3, 0.5],
            scaling=[0.8, 1.0],
            ltrb=True)
        self.slice = ops.Slice(device="gpu")

    def define_graph(self):
        rng = self.coin()
        rng2 = self.coin2()

        inputs, bboxes, labels = self.input()
        images = self.decode(inputs)

        # Paste and BBoxPaste need to use same scales and positions
        ratio = self.paste_ratio()
        px = self.paste_pos()
        py = self.paste_pos()
        images = self.paste(images, paste_x=px, paste_y=py, ratio=ratio)
        bboxes = self.bbpaste(bboxes, paste_x=px, paste_y=py, ratio=ratio)

        crop_begin, crop_size, bboxes, labels = self.prospective_crop(bboxes, labels)
        images = self.slice(images, crop_begin, crop_size)

        images = self.flip(images, horizontal=rng, vertical=rng2)
        bboxes = self.bbflip(bboxes, horizontal=rng, vertical=rng2)

        return (images, bboxes, labels)

In [25]:
start = time()
pipes = [COCOPipeline(batch_size=batch_size, num_threads=2, device_id=device_id)  for device_id in range(num_gpus)]
for pipe in pipes:
    pipe.build()
total_time = time() - start
print("Computation graph built and dataset loaded in %f seconds." % total_time)

Computation graph built and dataset loaded in 0.374428 seconds.


In [26]:
pipe_out = [pipe.run() for pipe in pipes]

images_cpu = pipe_out[0][0].as_cpu()
bboxes_cpu = pipe_out[0][1]
labels_cpu = pipe_out[0][2]

RuntimeError: Critical error in pipeline: [/opt/dali/dali/util/local_file.cc:105] File mapping failed: ./data/coco_detections.hdf5/COCO_train2014_000000000009.jpg
Stacktrace (9 entries):
[frame 0]: /opt/conda/lib/python3.6/site-packages/nvidia/dali/libdali.so(+0x6984e) [0x7fc56ae7e84e]
[frame 1]: /opt/conda/lib/python3.6/site-packages/nvidia/dali/libdali.so(+0x168c64) [0x7fc56af7dc64]
[frame 2]: /opt/conda/lib/python3.6/site-packages/nvidia/dali/libdali.so(dali::FileStream::Open(std::string const&, bool)+0xfb) [0x7fc56af7098b]
[frame 3]: /opt/conda/lib/python3.6/site-packages/nvidia/dali/libdali_operators.so(+0x496190) [0x7fc56c630190]
[frame 4]: /opt/conda/lib/python3.6/site-packages/nvidia/dali/libdali_operators.so(+0x4d5fa8) [0x7fc56c66ffa8]
[frame 5]: /opt/conda/lib/python3.6/site-packages/nvidia/dali/libdali_operators.so(+0x4d7205) [0x7fc56c671205]
[frame 6]: /opt/conda/lib/python3.6/site-packages/nvidia/dali/libdali_operators.so(+0xe6e8f0) [0x7fc56d0088f0]
[frame 7]: /lib/x86_64-

In [ ]:
img_index = 4

bboxes = bboxes_cpu.at(4)
bboxes

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random

img = images_cpu.at(img_index)

H = img.shape[0]
W = img.shape[1]
fig,ax = plt.subplots(1)

ax.imshow(img)
bboxes = bboxes_cpu.at(img_index)
labels = labels_cpu.at(img_index)
categories_set = set()
for label in labels:
    categories_set.add(label[0])

category_id_to_color = dict(
    [(cat_id, [random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]) for cat_id in categories_set])

for bbox, label in zip(bboxes, labels):
    rect = patches.Rectangle(
        (bbox[0] * W, bbox[1] * H), # Absolute corner coordinates
        (bbox[2] - bbox[0]) * W,    # Absolute bounding box width
        (bbox[3] - bbox[1]) * H,    # Absolute bounding box height
        linewidth=1,
        edgecolor=category_id_to_color[label[0]],
        facecolor='none')
    ax.add_patch(rect)

plt.show()

In [5]:
# Pipeline for image regions
image_field = ImageDetectionsField(detections_path=args.features_path, max_detections=50, load_in_tmp=False)

# Pipeline for text
text_field = TextField(init_token='<bos>', eos_token='<eos>', lower=True, tokenize='spacy',
                   remove_punctuation=True, nopoints=False)

In [7]:
# Create the dataset
dataset = COCO(image_field, text_field, 'coco/images/', args.annotation_folder, args.annotation_folder)
train_dataset, val_dataset, test_dataset = dataset.splits

NameError: name 'size' is not defined

In [8]:
type(train_dataset)

data.dataset.PairedDataset

In [ ]:
dataset.